In [1]:
%run sources/slicer.py
%run sources/time_series_to_intervals.py
%run sources/time_series_event_merge.py

In [2]:
main_value = "p01"
main_feature = "participant"
time_feature = "vt"

timeseries_df_list = {"paths": ["processed_data/fitbit/calories.csv",
                                "processed_data/fitbit/distance.csv",
                                "processed_data/fitbit/steps.csv",
                                "processed_data/fitbit/heart_rate.csv"],
                       "group_features": ["calories", "distance", "steps", "bpm"],
                       "bucket_values": [[1.16, 1.32, 1.59], [1236], [15.9], [51, 63, 76]]}

event_df_list = {"paths": ["processed_data/fitbit/exercise.csv",
                            "processed_data/fitbit/sleep_score.csv",
                            "processed_data/fitbit/sleep.csv"],
                  "group_features": ["Activity", "label", "sleep_phase"],
                  "value_feature": ["value", "value", "value"]}

slicer_params = {"stride": 24, "ow": 20, "ww": 4, "pw": 48, "granularity": "hours"}

final_odf, final_pdf = samples_from_dataset(main_value, main_feature, time_feature, timeseries_df_list, event_df_list, slicer_params)

100%|██████████| 149/149 [00:00<00:00, 1139.73it/s]


In [9]:
def sleep_score_from_prediction_window(dfs):
    r =[]
    for df in final_pdf["label"]:
        scs = df.to_dict(orient="records")
        scs.sort(key=lambda x: x['vt'])
        r.append(scs[0]['value'])    
    return pd.DataFrame({'SleepScore':r})

In [12]:
scores = sleep_score_from_prediction_window(final_pdf)
scores.describe()

,SleepScore
count,149.000000
mean,71.510067
std,5.651617
min,53.000000
25%,68.000000
50%,72.000000
75%,75.000000
max,88.000000


In [18]:
def score_buckets(s, buckets):
    for i in range(len(buckets)):
        if s <= buckets[i]:
            if i == 0 or s > buckets[i-1]:
                return i 
    return len(buckets)    


scores["class"] = scores.SleepScore.apply(lambda x: score_buckets(x,[68,72,75]))

In [19]:
scores

,SleepScore,class
0,79,3
1,71,1
2,68,0
3,73,2
4,67,0
...,...,...
144,71,1
145,76,3
146,72,1
147,67,0
